### MULTI-CLASS CLASSIFICATION
Implementation of multi scale classification using logistic regression with Neural Network architecture 

In [1]:
# importing python libraries 
import numpy as np
import pandas as pd
from scipy.io import loadmat
import matplotlib.pyplot as plt

### importing dataset 

In [2]:
#data preprocessing
filename1="./datasets/mnist_data.mat"
data = loadmat(filename1)
data.keys()

filename2="./datasets/mnist_weights.mat"
weights = loadmat(filename2)
weights.keys()

dict_keys(['__header__', '__version__', '__globals__', 'Theta1', 'Theta2'])

### Analysis from the dataset
Since the dataset is obtained from images which are handwritten number 
first row is our output. 
remaning rows contain info about the image.

In [3]:
Y = data['y']
# Add constant for intercept
X = np.c_[np.ones((data['X'].shape[0],1)), data['X']]
theta1, theta2 = weights['Theta1'], weights['Theta2']
n=X.shape[1]-1
m=Y.size
print("The number of features in our model (n):",n)
print("The number of training examples (m):",m)

The number of features in our model (n): 400
The number of training examples (m): 5000


### Visualization of dataset
For better understanding about the datset

In [4]:
no=20
sample = np.random.choice(X.shape[0], no)
plt.imshow(X[sample,1:].reshape(-1,no).T)
plt.axis('off');

In [5]:
def sigmoid(z):
    g=1/(1+np.exp(-z))
    return g

def cost_function(weights,X,Y,lamda):
    m = Y.size
    h = sigmoid(X.dot(weights))
    
    J = -1*(1/m)*(np.log(h).T.dot(Y)+np.log(1-h).T.dot(1-Y)) + (lamda/(2*m))*np.sum(np.square(weights[1:]))
    
    return(J[0])

def gradient(weights, X,Y,lamda):
    m = Y.size
    h = sigmoid(X.dot(weights.reshape(-1,1)))
      
    grad = (1/m)*X.T.dot(h-Y) + (lamda/m)*np.r_[[[0]],weights[1:].reshape(-1,1)]
        
    return(grad.flatten())

In [6]:
from scipy.optimize import minimize
def oneVsAll(X, Y, n_labels, lamda):
    initial_weights = np.zeros((X.shape[1],1))  
    all_weights = np.zeros((n_labels, X.shape[1])) 

    for i in np.arange(1, n_labels+1):
        model_parameters = minimize(cost_function, initial_weights, args=( X,(Y == i)*1,lamda), method=None,
                       jac=gradient, options={'maxiter':50})
        all_weights[i-1] = model_parameters.x
    return(all_weights)

In [7]:
final_parameters = oneVsAll(X, Y, 10, 0.1)

In [8]:
def predict(X,parameters):
    y = sigmoid(X.dot(parameters.T)).T
        
    return (np.argmax(y, axis=1)+1)

In [9]:
my_model_pred=predict(final_parameters,X)
print('Training set accuracy of my model: {} %'.format(np.mean(my_model_pred == Y.ravel())*100))

Training set accuracy of my model: 93.24 %


### Comparing my implemetation with sci-kit learn library

In [10]:
from sklearn.linear_model import LogisticRegression
sk_model = LogisticRegression(C=10, penalty='l1', solver='liblinear')
sk_model.fit(X[:,1:],Y.ravel())

LogisticRegression(C=10, penalty='l1', solver='liblinear')

### For better accuracy of the model we are using regularization techniques like lasso with lamda 10

In [11]:
sk_model_pred=sk_model.predict(X[:,1:])
print('Training set accuracy of scikit learn: {} %'.format(np.mean(sk_model_pred == Y.ravel())*100))

Training set accuracy of scikit learn: 96.96000000000001 %
